In [64]:
import pandas as pd
import os
import json
from concurrent.futures import ThreadPoolExecutor
import requests
import re

In [65]:
import selenium.webdriver as webdriver
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options as ChromeOptions
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException
from selenium.webdriver.support import expected_conditions as EC

from utilities.utilities import element_exists 

In [66]:
chrome_options = ChromeOptions()
service = ChromeService(executable_path=ChromeDriverManager().install())

In [67]:
# link where to download data
link = 'https://app.virtuals.io/virtuals/13015/contribution'

In [68]:
driver = webdriver.Chrome(options=chrome_options, service=service)
driver.get(link)
driver.maximize_window()
_ = WebDriverWait(driver, timeout=20).until(lambda driver: driver.execute_script('return document.readyState === "complete"'))

In [69]:
# /html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
# /html/body/div[1]/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
# /html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]
contribs_btn = driver.find_element(By.XPATH, '/html/body/div/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[2]/button[2]')
contribs_btn.click()

In [70]:
element_exists(driver, By.XPATH, '/html/body/path')

False

In [71]:
# select all div elements first
list_container = driver.find_element(By.XPATH, '/html/body/div[1]/div[1]/div/div/div/div[2]/div/div[2]/div/div/div[3]/div/div/div/div/div[1]')
divs = list_container.find_elements(By.CSS_SELECTOR, 'div.MuiStack-root')
divs_to_fine_tune = [div for div in divs if element_exists(div, By.CSS_SELECTOR, 'div:first-child > div:first-child > div:last-child') and div.find_element(By.CSS_SELECTOR, 'div:first-child > div:first-child > div:last-child').text.lower() == 'available']

# check each div elements have its decendant span.MuiChip-label if any 

In [72]:
for div in divs_to_fine_tune:
    print(f'{div.text}\n')

Dataset#7638
Available
New Data: RagmonDEX
0xAEB6...b984FD contributed 15 hours ago

Dataset#7638
Available
New Data: RagmonDEX
0xAEB6...b984FD contributed 15 hours ago

Dataset#7638
Available

Dataset#7636
Available
New Data: RagmonDeX
0xAEB6...b984FD contributed 16 hours ago

Dataset#7636
Available
New Data: RagmonDeX
0xAEB6...b984FD contributed 16 hours ago

Dataset#7636
Available

Dataset#7634
Available
New Data: RagmonDex
0xAEB6...b984FD contributed 18 hours ago

Dataset#7634
Available
New Data: RagmonDex
0xAEB6...b984FD contributed 18 hours ago

Dataset#7634
Available

Dataset#7622
Available
New Data: Ragnarok Monster DataDex
0xAEB6...b984FD contributed 19 hours ago

Dataset#7622
Available
New Data: Ragnarok Monster DataDex
0xAEB6...b984FD contributed 19 hours ago

Dataset#7622
Available

Model#7617
Available
meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
0xAEB6...b984FD contributed 20 hours ago

Model#7617
Available
meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
0xAEB6...b984FD cont

In [73]:
outputs = []
for link_container in divs_to_fine_tune:
    print(link_container.text)
    if element_exists(link_container, By.CSS_SELECTOR, 'a'):
        a_tag = link_container.find_element(By.CSS_SELECTOR, 'a')
        dataset_num = link_container.find_element(By.CSS_SELECTOR, 'span.MuiChip-label')
        outputs.append((dataset_num, a_tag))

Dataset#7638
Available
New Data: RagmonDEX
0xAEB6...b984FD contributed 15 hours ago
Dataset#7638
Available
New Data: RagmonDEX
0xAEB6...b984FD contributed 15 hours ago
Dataset#7638
Available
Dataset#7636
Available
New Data: RagmonDeX
0xAEB6...b984FD contributed 16 hours ago
Dataset#7636
Available
New Data: RagmonDeX
0xAEB6...b984FD contributed 16 hours ago
Dataset#7636
Available
Dataset#7634
Available
New Data: RagmonDex
0xAEB6...b984FD contributed 18 hours ago
Dataset#7634
Available
New Data: RagmonDex
0xAEB6...b984FD contributed 18 hours ago
Dataset#7634
Available
Dataset#7622
Available
New Data: Ragnarok Monster DataDex
0xAEB6...b984FD contributed 19 hours ago
Dataset#7622
Available
New Data: Ragnarok Monster DataDex
0xAEB6...b984FD contributed 19 hours ago
Dataset#7622
Available
Model#7617
Available
meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
0xAEB6...b984FD contributed 20 hours ago
Model#7617
Available
meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo
0xAEB6...b984FD contributed 20 ho

In [74]:
outputs = list(set(outputs))
outputs

[(<selenium.webdriver.remote.webelement.WebElement (session="4793ce7688556b9f8e82ebdd8f1f3f76", element="f.87E10353477B684286BEEC3881721A2B.d.42BE8A2272E822AA9CAE48CB5E962C92.e.249")>,
  <selenium.webdriver.remote.webelement.WebElement (session="4793ce7688556b9f8e82ebdd8f1f3f76", element="f.87E10353477B684286BEEC3881721A2B.d.42BE8A2272E822AA9CAE48CB5E962C92.e.248")>),
 (<selenium.webdriver.remote.webelement.WebElement (session="4793ce7688556b9f8e82ebdd8f1f3f76", element="f.87E10353477B684286BEEC3881721A2B.d.42BE8A2272E822AA9CAE48CB5E962C92.e.257")>,
  <selenium.webdriver.remote.webelement.WebElement (session="4793ce7688556b9f8e82ebdd8f1f3f76", element="f.87E10353477B684286BEEC3881721A2B.d.42BE8A2272E822AA9CAE48CB5E962C92.e.256")>),
 (<selenium.webdriver.remote.webelement.WebElement (session="4793ce7688556b9f8e82ebdd8f1f3f76", element="f.87E10353477B684286BEEC3881721A2B.d.42BE8A2272E822AA9CAE48CB5E962C92.e.263")>,
  <selenium.webdriver.remote.webelement.WebElement (session="4793ce768855

In [77]:
for dataset_num, a_tag in outputs:
    link = a_tag.get_attribute('href')
    # new_file_name = re.sub(r'')
    print(f'{dataset_num.text} {a_tag.get_attribute('href')}')
    file_name = link.split('/')[-1]
    print(file_name)

Model#7512 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7512/ROronXT.json
ROronXT.json
Model#7470 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7470/RORonAI.json
RORonAI.json
Model#7462 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7462/X%20Update%20Aggregator%20AI%20Model.json
X%20Update%20Aggregator%20AI%20Model.json
Model#7591 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7591/character.json
character.json
Model#7277 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7277/character.json
character.json
Model#7479 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7479/RORonXAI.json
RORonXAI.json
Model#7491 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7491/character.json
character.json
Model#7469 https://vpmodels-prod.s3.ap-southeast-1.amazonaws.com/contribution-7469/ROronAI.json
ROronAI.json
Model#7617 https://vpmodels-prod.s3.ap-southeast-1.amazo

In [78]:
output_dir = './ROron-ron input'
os.makedirs(output_dir, exist_ok=True)

In [82]:
for dataset_num, a_tag in outputs:
    # extract href
    link = a_tag.get_attribute('href')

    # get file name to when link is downloaded
    file_name = link.split('/')[-1]

    number = dataset_num.text

    r = requests.get(link, allow_redirects=True)
    with open(f'{output_dir}/{number}{file_name}', 'wb') as file:
        file.write(r.content)